In [1]:
import ccxt

In [33]:
exchange = ccxt.binance({
    'apiKey': 'q0HAhhxXOJr8HBbmYFYe0QMhPMrbfWVGngOUeccqXQHdgCk5fKqq97vfykJm8EGk',
    'secret': 'LSkB2x2lO8r5fx6qaTR4rGObGFjRHbJHwHPbpSY4LQ8Hp9uU7vKaueVaDavbVA3C',
    'options': {'defaultType': 'future'},
})

In [34]:
exchange = ccxt.binance({
    'apiKey': 'q0HAhhxXOJr8HBbmYFYe0QMhPMrbfWVGngOUeccqXQHdgCk5fKqq97vfykJm8EGk',
    'secret': 'LSkB2x2lO8r5fx6qaTR4rGObGFjRHbJHwHPbpSY4LQ8Hp9uU7vKaueVaDavbVA3C',
    'options': {
        'defaultType': 'future',  # Use Futures endpoints
    }
})
exchange.enableDemoTrading(True)  # Enable demo trading mode


In [10]:
exchange = ccxt.binance({
    'apiKey': 'eOUQOwsdCjiNMUDbNsrTV0TUPPF7rAiEXiWtk5CMEGClviPoqBqFhm1KFddUtyoE',
    'secret': 'rpPI79M1aoh8sc7oeZJTj8Fox3ppcBPhtBvWXtTBCWW65q4Ys23tBxo5MZ8tgIZv',
    'options': {
        'defaultType': 'future',  # Use Futures endpoints
    },
    'urls': {
        'api': {
            'fapiPublic': 'https://testnet.binancefuture.com/fapi/v1',
            'fapiPrivate': 'https://testnet.binancefuture.com/fapi/v1',
        }
    }
})
exchange.enable_demo_trading(True)  # Enable demo trading mode


In [11]:
symbol = "BTC/USDT"
amount = 0.001

In [15]:
from decimal import Decimal, ROUND_DOWN

balance = 100.0
open_positions = []   # each: {id, side: 'long'/'short', entry, qty, notional(size_usd), grid_id, created_at, tp_price, sl_price, last_price}
open_orders = []      # each: {id, grid_id, side, price, size_usd, status, ccxt_id, tp, sl}
grids = {}            # grid metadata

def usd_to_qty(usd, price):
    return usd / price

def round_amount_qty(qty, symbol_local=None):
    """
    Round quantity according to market precision and ensure it's >= exchange min amount.
    Returns a float suitable for order placement.
    """
    try:
        symbol_send = symbol_local if symbol_local else symbol
        market = exchange.market(symbol_send)
        
        # Get amount precision
        prec = int(market.get('precision', {}).get('amount', 6))
        
        # Get min amount
        min_amount = float(market.get('limits', {}).get('amount', {}).get('min', 10**(-prec)))
        
        # Round down with Decimal
        quant = Decimal(str(qty)).quantize(Decimal('1e-{}'.format(prec)), rounding=ROUND_DOWN)
        qty_rounded = max(Decimal(str(min_amount)), quant)
        
        # Safety: check if rounded qty is valid
        if qty_rounded < Decimal(str(min_amount)):
            print(f"❌ Qty {qty_rounded} below min {min_amount}, skipping order.")
            return 0.0
        
        return float(qty_rounded)
    
    except Exception as e:
        print(f"[ROUND_AMOUNT_QTY ERROR] {e}")
        # fallback: simple round with 6 decimals
        return float(round(qty, 6))


In [18]:
symbol = "BTC/USDT"
amount = 0.001

def place_market_order_ccxt(side, amount, pos_side=None, reduce_only=False, symbol_local=None):
    """
    Unified market order helper for futures (demo/live) in hedge mode.
        side: 'buy' or 'sell' (action)
        amount: already-calculated asset amount (float)
        pos_side: 'LONG' or 'SHORT' (required in hedge mode)
        reduce_only: boolean; include reduceOnly param only when True
        symbol_local: optional symbol string (e.g., "BTC/USDT") to use for precision rounding
    Returns a dict-like result (never None) with keys including 'id','amount','status','error' on failure.
    """
    symbol_send = symbol_local if symbol_local else symbol  # use global symbol fallback
    amount = round_amount_qty(amount, symbol_send)
    if amount <= 0:
        return {'id': None, 'amount': 0, 'status': 'failed', 'error': 'amount<=0 after rounding'}

    params = {}
    # Always include positionSide in hedge mode
    if pos_side:
        params['positionSide'] = pos_side

    # Only add reduceOnly if explicitly requested
    if reduce_only:
        params['reduceOnly'] = True

    try:
        # Market order: price=None
        order = exchange.create_order(symbol_send, 'market', side, amount, None, params)
        # Normalize response
        return {
            'id': order.get('id') if isinstance(order, dict) else getattr(order, 'id', None),
            'average': float(order.get('average', 0)) if isinstance(order, dict) else float(getattr(order, 'average', 0)),
            'amount': float(order.get('filled', order.get('amount', amount))) if isinstance(order, dict) else amount,
            'status': order.get('status', 'closed') if isinstance(order, dict) else 'closed',
            'raw': order
        }
    except Exception as e:
        print(f"❌ Order placement failed: {e}")
        return {'id': None, 'amount': 0, 'status': 'failed', 'error': str(e)}


In [24]:
close_pos = place_market_order_ccxt('buy', amount, pos_side='LONG')
print (close_pos)

{'id': '6733189516', 'average': 109382.6, 'amount': 0.001, 'status': 'closed', 'raw': {'info': {'orderId': 6733189516, 'symbol': 'BTCUSDT', 'status': 'FILLED', 'clientOrderId': 'x-cvBPrNm98aa2f67f327210e597016d', 'price': '0.00', 'avgPrice': '109382.60000', 'origQty': '0.001', 'executedQty': '0.001', 'cumQty': '0.001', 'cumQuote': '109.38260', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'LONG', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1761217474878}, 'id': '6733189516', 'clientOrderId': 'x-cvBPrNm98aa2f67f327210e597016d', 'timestamp': 1761217474878, 'datetime': '2025-10-23T11:04:34.878Z', 'lastTradeTimestamp': 1761217474878, 'lastUpdateTimestamp': 1761217474878, 'symbol': 'BTC/USDT:USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': F

In [9]:
close_pos2 = place_market_order_ccxt('sell', amount, pos_side='LONG')
print (close_pos2)

❌ Order placement failed: binance {"code":-2022,"msg":"ReduceOnly Order is rejected."}
{'id': None, 'amount': 0, 'status': 'failed', 'error': 'binance {"code":-2022,"msg":"ReduceOnly Order is rejected."}'}


In [29]:
import time

# --- Setup ---
last_price = float(exchange.fetch_ticker(symbol)['last'])
print(f"Last price: {last_price} USDT")

amount = 0.001  # size in base asset (BTC, ETH, etc.)

# Place initial SHORT (sell) market order
ccxt_order = exchange.create_order(
    symbol=symbol,
    type='market',
    side='sell',
    amount=amount,
    params={'positionSide': 'SHORT'}
)
entry_price = float(ccxt_order['average'] or ccxt_order['price'])
print(f"SHORT opened at {entry_price} USDT")

# Store initial balance
balance_data = exchange.fetch_balance(params={'type': 'future'})
initial_balance = float(balance_data['total'].get('USDT', 0))
print(f"Initial demo balance: {initial_balance} USDT")

# Define TP and SL in USD
take_profit_diff = 100  # how far price must fall for TP
stop_loss_diff = 50     # how far price must rise for SL

tp_price = entry_price - take_profit_diff
sl_price = entry_price + stop_loss_diff
print(f"Take Profit at {tp_price}, Stop Loss at {sl_price}")

# --- Monitoring loop ---
while True:
    current_price = float(exchange.fetch_ticker(symbol)['last'])
    print(f"Current price: {current_price} USDT")

    # Take profit
    if current_price <= tp_price:
        close_order = exchange.create_order(
            symbol=symbol,
            type='market',
            side='buy',
            amount=amount,
            params={'positionSide': 'SHORT'}
        )
        print(f"[TP HIT] SHORT closed at {current_price} USDT")
        break

    # Stop loss
    elif current_price >= sl_price:
        close_order = exchange.create_order(
            symbol=symbol,
            type='market',
            side='buy',
            amount=amount,
            params={'positionSide': 'SHORT'}
        )
        print(f"[SL HIT] SHORT closed at {current_price} USDT")
        break

    else:
        print("Holding position, no TP/SL hit.")

    print("-------------------------------------")
    time.sleep(2)

# --- Final balance fetch ---
balance_data = exchange.fetch_balance(params={'type': 'future'})
final_balance = float(balance_data['total'].get('USDT', 0))
profit_loss = final_balance - initial_balance
print(f"Final demo balance: {final_balance} USDT")
print(f"Profit/Loss from trade: {profit_loss} USDT")


Last price: 109017.7 USDT
SHORT opened at 108980.2 USDT
Initial demo balance: 4950.24311192 USDT
Take Profit at 108880.2, Stop Loss at 109030.2
Current price: 109017.7 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109017.7 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109019.7 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109019.7 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109028.3 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109028.3 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109034.8 USDT
[SL HIT] SHORT closed at 109034.8 USDT
Final demo balance: 4950.19715861 USDT
Profit/Loss from trade: -0.04595331000018632 USDT


In [8]:
import uuid
from datetime import datetime

position_fraction = 0.1  # fraction of balance per grid position
grid_levels = 10        # total number of grid levels
min_tp_pct = 0.002      # minimum take-profit percentage (0.2%)
user_grid_spacing = 50.0  # fixed spacing in USD between grid levels    
def place_grid(grid_direction, center_price, mode='trend', fraction=position_fraction):
    """
    Symmetric ladder grid system:
    - For 'up' we create buy levels (long entries)
    - For 'down' we create sell levels (short entries)
    """
    gid = str(uuid.uuid4())[:8]
    grids[gid] = {
        'id': gid,
        'direction': grid_direction,
        'active': True,
        'created_at': datetime.utcnow(),
        'orders': [],
        'frozen': False
    }

    size_usd = max(20.0, balance * fraction)

    # Generate grid prices using user-defined spacing
    half = grid_levels // 2
    grid_prices = [center_price + i * user_grid_spacing for i in range(-half, half + 1)]
    grid_prices = sorted(set(grid_prices))

    # for down (short grid) we use 'sell' entries, for up we use 'buy'
    order_side = 'sell' if grid_direction == 'down' else 'buy'

    for i, level_price in enumerate(grid_prices):
    # Handle Take-Profit (TP)
        if grid_direction == 'up':
            # TP should be the next higher grid (unless it's the last one)
            if i + 1 < len(grid_prices):
                tp_price = grid_prices[i + 1]
            else:
                tp_price = level_price * (1 + min_tp_pct)
            # SL slightly below the current level
            sl_price = level_price - 0.5 * user_grid_spacing

        else:  # grid_direction == 'down'
            # TP should be the next lower grid (unless it's the first one)
            if i > 0:
                tp_price = grid_prices[i - 1]
            else:
                tp_price = level_price * (1 - min_tp_pct)
            # SL slightly above the current level
            sl_price = level_price + 0.5 * user_grid_spacing    
        """try:
            #resp = place_limit_order_ccxt(order_side, level_price, size_usd)
            #ccxt_id = resp.get('id')
        except Exception:
            #ccxt_id = None"""

        oid = str(uuid.uuid4())[:10]
        order = {
            'id': oid,
            'grid_id': gid,
            'side': order_side,
            'price': float(level_price),
            'size_usd': float(size_usd),
            'status': 'open',
            'tp': float(tp_price),
            'sl': float(sl_price)
        }
        open_orders.append(order)
        grids[gid]['orders'].append(oid)

    print(f"[GRID PLACED] id={gid}, dir={grid_direction}, mode={mode}, levels={len(grid_prices)}")
    print("------------------------------------------------------------")
    print(f"[DEBUG] Grid Details for {gid}:")
    for i, level_price in enumerate(grid_prices):
        # Handle Take-Profit (TP)
        if grid_direction == 'up':
            # TP should be the next higher grid (unless it's the last one)
            if i + 1 < len(grid_prices):
                tp_price = grid_prices[i + 1]
            else:
                tp_price = level_price * (1 + min_tp_pct)
            # SL slightly below the current level
            sl_price = level_price - 0.5 * user_grid_spacing

        else:  # grid_direction == 'down'
            # TP should be the next lower grid (unless it's the first one)
            if i > 0:
                tp_price = grid_prices[i - 1]
            else:
                tp_price = level_price * (1 - min_tp_pct)
            # SL slightly above the current level
            sl_price = level_price + 0.5 * user_grid_spacing
        print(f"Grid {i+1}/{len(grid_prices)} | Price: {level_price:.2f} | TP: {tp_price:.2f} with difference = {tp_price - level_price} | SL: {sl_price:.2f} with difference = {sl_price - level_price}")
    print("------------------------------------------------------------")
    return gid

place_grid('up', 30000.0, mode='trend')



[GRID PLACED] id=71d8b8cc, dir=up, mode=trend, levels=11
------------------------------------------------------------
[DEBUG] Grid Details for 71d8b8cc:
Grid 1/11 | Price: 29750.00 | TP: 29800.00 with difference = 50.0 | SL: 29725.00 with difference = -25.0
Grid 2/11 | Price: 29800.00 | TP: 29850.00 with difference = 50.0 | SL: 29775.00 with difference = -25.0
Grid 3/11 | Price: 29850.00 | TP: 29900.00 with difference = 50.0 | SL: 29825.00 with difference = -25.0
Grid 4/11 | Price: 29900.00 | TP: 29950.00 with difference = 50.0 | SL: 29875.00 with difference = -25.0
Grid 5/11 | Price: 29950.00 | TP: 30000.00 with difference = 50.0 | SL: 29925.00 with difference = -25.0
Grid 6/11 | Price: 30000.00 | TP: 30050.00 with difference = 50.0 | SL: 29975.00 with difference = -25.0
Grid 7/11 | Price: 30050.00 | TP: 30100.00 with difference = 50.0 | SL: 30025.00 with difference = -25.0
Grid 8/11 | Price: 30100.00 | TP: 30150.00 with difference = 50.0 | SL: 30075.00 with difference = -25.0
Grid 9/

/tmp/ipykernel_5257/589931995.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'created_at': datetime.utcnow(),


'71d8b8cc'

In [26]:
initial_balance = exchange.fetch_balance(params={'type': 'future'})['total']['USDT']
def compute_realistic_profit(symbol):
    info = exchange.fetch_balance(params={'type': 'future'})['info']
    wallet_balance = float(info['totalWalletBalance'])       # includes realized PnL
    unrealized = float(info.get('totalUnrealizedProfit', 0)) # safety fallback

    simulated_balance = wallet_balance + unrealized
    net_profit = simulated_balance - initial_balance
    return simulated_balance, net_profit, unrealized, wallet_balance
simulated_balance, net_profit, unrealized, wallet_balance = compute_realistic_profit(symbol)
print(f"Simulated Balance: {simulated_balance:.2f} USDT, Net Profit: {net_profit:.2f} USDT, unrealized PnL: {unrealized:.2f} USDT")
print(f"Wallet Balance: {wallet_balance:.2f} USDT, Unrealized PnL: {unrealized:.2f} USDT"   )

Simulated Balance: 4952.39 USDT, Net Profit: -0.00 USDT, unrealized PnL: -0.03 USDT
Wallet Balance: 4952.42 USDT, Unrealized PnL: -0.03 USDT


In [31]:
import time

# --- Setup ---
last_price = float(exchange.fetch_ticker(symbol)['last'])
print(f"Last price: {last_price} USDT")

amount = 0.001  # size in base asset (BTC, ETH, etc.)

# Place initial SHORT (sell) market order using your wrapper
ccxt_order = place_market_order_ccxt(
    side='sell',
    amount=amount,
    pos_side='SHORT'
)
entry_price = float(ccxt_order['average'] or ccxt_order.get('price', last_price))
print(f"SHORT opened at {entry_price} USDT")

# Store initial balance
balance_data = exchange.fetch_balance(params={'type': 'future'})
initial_balance = float(balance_data['total'].get('USDT', 0))
print(f"Initial demo balance: {initial_balance} USDT")

# Define TP and SL in USD
take_profit_diff = 100  # how far price must fall for TP
stop_loss_diff = 50     # how far price must rise for SL

tp_price = entry_price - take_profit_diff
sl_price = entry_price + stop_loss_diff
print(f"Take Profit at {tp_price}, Stop Loss at {sl_price}")

# --- Monitoring loop ---
while True:
    current_price = float(exchange.fetch_ticker(symbol)['last'])
    print(f"Current price: {current_price} USDT")

    # Take profit
    if current_price <= tp_price:
        close_order = place_market_order_ccxt(
            side='buy',
            amount=amount,
            pos_side='SHORT'
        )
        print(f"[TP HIT] SHORT closed at {current_price} USDT")
        break

    # Stop loss
    elif current_price >= sl_price:
        close_order = place_market_order_ccxt(
            side='buy',
            amount=amount,
            pos_side='SHORT'
        )
        print(f"[SL HIT] SHORT closed at {current_price} USDT")
        break

    else:
        print("Holding position, no TP/SL hit.")

    print("-------------------------------------")
    time.sleep(2)

# --- Final balance fetch ---
balance_data = exchange.fetch_balance(params={'type': 'future'})
final_balance = float(balance_data['total'].get('USDT', 0))
profit_loss = final_balance - initial_balance
print(f"Final demo balance: {final_balance} USDT")
print(f"Profit/Loss from trade: {profit_loss} USDT")


Last price: 109069.5 USDT
SHORT opened at 109061.7 USDT
Initial demo balance: 4950.03210121 USDT
Take Profit at 108961.7, Stop Loss at 109111.7
Current price: 109069.5 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109068.2 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109068.2 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109068.2 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109069.6 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109069.6 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109080.7 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109080.7 USDT
Holding position, no TP/SL hit.
-------------------------------------
Current price: 109074.0 USDT
Holding position, no TP/SL hit.
---